<a href="https://colab.research.google.com/github/tthuy123/graduation/blob/main/CNN%2BLSTM_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

so this one code is re-implementation/adopted from this paper, i techinacally first follow their method to see if the result works. After that i will see hehehehehe


In [2]:
import pandas as pd

In [3]:
df_info = pd.read_csv("/content/drive/MyDrive/OULAD/studentInfo.csv")
df_vle = pd.read_csv("/content/drive/MyDrive/OULAD/studentVle.csv")
df_assess = pd.read_csv("/content/drive/MyDrive/OULAD/studentAssessment.csv")
df_assessments = pd.read_csv("/content/drive/MyDrive/OULAD/assessments.csv")
df_courses = pd.read_csv("/content/drive/MyDrive/OULAD/courses.csv")
df_regis = pd.read_csv("/content/drive/MyDrive/OULAD/studentRegistration.csv")

In [ ]:
df_assessments

In [ ]:
display(df_assessments['assessment_type'].unique())

array(['TMA', 'Exam', 'CMA'], dtype=object)

In [ ]:
df_assess

In [ ]:
df_vle

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


In [ ]:
df_courses

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240
5,BBB,2014B,234
6,CCC,2014J,269
7,CCC,2014B,241
8,DDD,2013J,261
9,DDD,2014J,262


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

In [5]:
## Merge data
vle_agg = df_vle.groupby(['id_student', 'code_module', 'code_presentation'])['sum_click'].sum().reset_index()
vle_agg.rename(columns={'sum_click': 'total_clicks'}, inplace=True)
vle_agg


,id_student,code_module,code_presentation,total_clicks
0,6516,AAA,2014J,2791
1,8462,DDD,2013J,646
2,8462,DDD,2014J,10
3,11391,AAA,2013J,934
4,23629,BBB,2013B,161
...,...,...,...,...
29223,2698257,AAA,2013J,758
29224,2698535,CCC,2014B,786
29225,2698535,EEE,2013J,3455
29226,2698577,BBB,2014J,717


In [ ]:
df_info

In [6]:
# 1. LOAD DỮ LIỆU & LÀM SẠCH CƠ BẢN
df_info['target'] = df_info['final_result'].apply(lambda x: 1 if x in ['Distinction', 'Pass'] else 0)
## df_info

In [7]:
# 2. XỬ LÝ DỮ LIỆU TIME-SERIES (VLE CLICKS)
# goal: tạo chuỗi lịch sử tương tác của học sinh theo các tuần
MAX_WEEKS = 40 # Chọn 40 vì dựa theo bảng courses max length là 269 ngày xấp xỉ 38.4 tuần
def map_date_to_week(date_val):
  if date_val < 0:
    return 0
  w = date_val // 7

  return w if w < MAX_WEEKS else MAX_WEEKS -1
df_vle['week_idx'] = df_vle['date'].apply(map_date_to_week)

week_activity = df_vle.groupby(['id_student', 'code_module', 'code_presentation', 'week_idx'])['sum_click'].sum().reset_index()

pivot_data = week_activity.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='sum_click',
    fill_value=0
)
expected_cols = list(range(MAX_WEEKS)) # [0, 1, ..., 39]
pivot_data = pivot_data.reindex(columns=expected_cols, fill_value=0)

pivot_data.columns = [f'week_{i}' for i in pivot_data.columns] # Đổi tên cho rõ ràng
pivot_data.reset_index(inplace=True)
pivot_data


,id_student,code_module,code_presentation,week_0,week_1,week_2,week_3,week_4,week_5,week_6,...,week_30,week_31,week_32,week_33,week_34,week_35,week_36,week_37,week_38,week_39
0,6516,AAA,2014J,485.0,42.0,79.0,193.0,69.0,34.0,10.0,...,55.0,54.0,119.0,79.0,67.0,0.0,0.0,0.0,2.0,0
1,8462,DDD,2013J,162.0,146.0,9.0,23.0,79.0,18.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,8462,DDD,2014J,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,11391,AAA,2013J,281.0,20.0,100.0,0.0,26.0,60.0,22.0,...,43.0,35.0,0.0,15.0,85.0,7.0,1.0,0.0,0.0,0
4,23629,BBB,2013B,37.0,5.0,9.0,2.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223,2698257,AAA,2013J,82.0,10.0,124.0,75.0,33.0,16.0,60.0,...,25.0,1.0,9.0,21.0,0.0,0.0,0.0,0.0,1.0,0
29224,2698535,CCC,2014B,178.0,217.0,6.0,0.0,19.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
29225,2698535,EEE,2013J,39.0,2.0,205.0,69.0,485.0,284.0,409.0,...,7.0,0.0,27.0,70.0,0.0,0.0,0.0,0.0,0.0,0
29226,2698577,BBB,2014J,0.0,0.0,49.0,23.0,96.0,150.0,12.0,...,3.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0


Nhận xét về CMA (Computer marked assessment) thấy rằng phần nhiều những bài assessment này có trọng số điểm bằng 0.
Mong muốn khai thác dữ liệu điểm số này theo hai hướng time-series chạy song song với chuỗi click ở phía trên.
1. Chuỗi điểm số: Tính điểm sinh viên đạt được trong 1 tuần
2. Chuỗi độ trễ: Sinh viên nộp sớm hay muộn bao nhiêu ngày.

Như vậy cấu trúc dữ liệu đầu vào mô hình Thay vì (Samples, 40, 1) chỉ có click, giờ ta sẽ có (Samples, 40, 3):
Channel 1: Số lượng Click (Activity)

Channel 2: Điểm số trung bình tuần đó (Performance)

Channel 3: Số ngày nộp sớm/muộn (Discipline)

In [9]:
# 3. XỬ LÝ DỮ LIỆU ASSESSMENT (ĐIỂM SỐ)
assess_full = pd.merge(df_assess, df_assessments, on='id_assessment', how='inner')

# 1. Xử lý score
assess_full['score'] = assess_full['score'].fillna(0) # score không có thì coi là bằng 0

# 2. Tính độ trễ
# Công thức: Ngày nộp - Deadline
# Giá trị dương (+): Nộp muộn (Ví dụ: nộp ngày 30, deadline 25 -> +5)
# Giá trị âm (-): Nộp sớm (Ví dụ: nộp ngày 20, deadline 25 -> -5)
# Lưu ý: Cần xử lý NaN ở cột 'date' (deadline) nếu có.
assess_full['lateness'] = assess_full['date_submitted'] - assess_full['date']
assess_full['lateness'] = assess_full['lateness'].fillna(0) # Nếu không có deadline thì coi như đúng hạn

# 3. Map sang tuần (week index) dựa trên date_submitted
def map_to_week(day):
    if pd.isna(day) or day < 0: return 0
    w = int(day // 7)
    return min(w, 39) # Kẹp trần ở tuần 39

assess_full['week_idx'] = assess_full['date_submitted'].apply(map_to_week)

# 2. Tạo pivot table
# 2.1 Pivot bảng điểm (Score Series)
# Nếu 1 tuần nộp nhiều bài -> Tính trung bình điểm tuần đó
score_pivot = assess_full.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='score',
    aggfunc='mean',
    fill_value=-1
    # QUAN TRỌNG: Fill -1 (hoặc 0) cho những tuần KHÔNG có bài kiểm tra.
    # Ở đây tôi chọn -1 để mô hình phân biệt được "được 0 điểm" và "không có bài".
    # Tuy nhiên, nếu dùng ReLU activation sau này thì nên để 0. Ta tạm để 0 cho an toàn với CNN.
).fillna(0)

# 2.2 Pivot bảng độ trễ (Lateness Series)
lateness_pivot = assess_full.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='lateness',
    aggfunc='mean', # Trung bình độ trễ các bài trong tuần
    fill_value=0 # Không nộp bài tuần đó -> coi như không trễ (0)
).fillna(0)

# 2.3 Pivot Weight (Trọng số) - QUAN TRỌNG
# Đây là kênh thông tin mới: Cho biết tuần đó có bài quan trọng không?
# weight_pivot = assess_full.pivot_table(
#     index=['id_student', 'code_module', 'code_presentation'],
#     columns='week_idx',
#     values='weight',
#     aggfunc='max', # Nếu tuần đó có nhiều bài, lấy bài có trọng số cao nhất
#     fill_value=0
# )

# --- BƯỚC 3: ĐỒNG BỘ HÓA (PADDING/REINDEX) ---
# Đảm bảo đủ 40 tuần (từ week_0 đến week_39) giống bảng Click
expected_cols = list(range(40))

# Hàm reindex chuẩn hóa
def align_columns(df, prefix):
    df = df.reindex(columns=expected_cols, fill_value=0)
    df.columns = [f'{prefix}_{i}' for i in df.columns]
    return df.reset_index()

df_score_ts = align_columns(score_pivot, 'score_wk')
df_lateness_ts = align_columns(lateness_pivot, 'late_wk')
# df_weight_ts = align_columns(weight_pivot, 'weight_wk') # Chuỗi thời gian của trọng số




In [ ]:
df_score_ts

In [ ]:
df_lateness_ts

In [13]:
# 4. Gộp dữ liệu
df_merged = df_info.copy()

# merge với dữ liệu click
df_merged = pd.merge(df_merged, pivot_data, on=['id_student', 'code_module', 'code_presentation'], how='left')
# merge với dữ liệu điểm số
df_merged = pd.merge(df_merged, df_score_ts, on=['id_student', 'code_module', 'code_presentation'], how='left')
# merge với dữ liệu độ trễ
df_merged = pd.merge(df_merged, df_lateness_ts, on=['id_student', 'code_module', 'code_presentation'], how='left')

df_merged.fillna(0, inplace=True) # Fill NaN bằng 0

print(f"Kích thước sau khi merge: {df_merged.shape}")

Kích thước sau khi merge: (32593, 133)


In [16]:
# 5 Feature Engineering

# Xác định các nhóm cột
ts_click_cols = [f'week_{i}' for i in range(40)]
ts_score_cols = [f'score_wk_{i}' for i in range(40)]
ts_late_cols  = [f'late_wk_{i}' for i in range(40)]

all_ts_cols = ts_click_cols + ts_score_cols + ts_late_cols

# Xác định các cột static
exclude_cols = ['id_student', 'code_module', 'code_presentation', 'final_result', 'target', 'date_registration', 'date_unregistration'] + all_ts_cols
static_cols = [c for c in df_merged.columns if c not in exclude_cols]
print(f"Các cột Static được sử dụng: {static_cols}")

# Encode static sử dụng one-hot encoder
# Với các biến có thử tự sử dụng ordinal encoding
# 5.3 Áp dụng Ordinal Encoding (Xử lý biến thứ bậc TRƯỚC)
ordinal_mappings = {
    "age_band": {"0-35": 0.0, "35-55": 0.5, "55<=": 1.0},
    "gender": {"M": 0.0, "F": 1.0},
    "highest_education": {
        "No Formal quals": 0.0,
        "Lower Than A Level": 0.25,
        "A Level or Equivalent": 0.5,
        "HE Qualification": 0.75,
        "Post Graduate Qualification": 1.0,
    },
    "imd_band": {
        "0-10%": 0.0, "10-20%": 0.1, "20-30%": 0.2, "30-40%": 0.3,
        "40-50%": 0.4, "50-60%": 0.5, "60-70%": 0.6, "70-80%": 0.7,
        "80-90%": 0.8, "90-100%": 0.9, "?": 0.5
    }
}

# Tạo bản sao để xử lý
df_encoded = df_merged.copy()

# Duyệt qua dictionary và map dữ liệu
for col, mapping in ordinal_mappings.items():
    if col in df_encoded.columns:
        df_encoded[col] = df_encoded[col].map(mapping)
        # Fill những giá trị không map được (nếu có) bằng 0.5 hoặc giá trị mặc định
        df_encoded[col] = df_encoded[col].fillna(0.5)

# 5.4 Áp dụng One-Hot Encoding (Cho các biến định danh còn lại)
# Lúc này age_band, gender... đã thành số (float), nên select_dtypes sẽ tự bỏ qua chúng
# Chỉ còn lại các cột như 'region', 'disability' v.v.
remaining_cat_cols = df_encoded[static_cols].select_dtypes(include=['object']).columns

print(f"Các cột sẽ One-Hot Encoding: {remaining_cat_cols}")

df_encoded = pd.get_dummies(df_encoded, columns=remaining_cat_cols, drop_first=True)

# 5.5 Cập nhật danh sách Static Columns cuối cùng
# Lấy tất cả feature trong df_encoded ngoại trừ feature bị loại bỏ và feature time-series
final_static_cols = [c for c in df_encoded.columns if c not in (exclude_cols + all_ts_cols + ['target'])]

print("-" * 30)
print(f"Số lượng Feature tĩnh cuối cùng: {len(final_static_cols)}")
print(f"Danh sách: {final_static_cols}")
print("-" * 30)

Các cột Static được sử dụng: ['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability']
Các cột sẽ One-Hot Encoding: Index(['region', 'disability'], dtype='object')
------------------------------
Số lượng Feature tĩnh cuối cùng: 19
Danh sách: ['gender', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'region_East Midlands Region', 'region_Ireland', 'region_London Region', 'region_North Region', 'region_North Western Region', 'region_Scotland', 'region_South East Region', 'region_South Region', 'region_South West Region', 'region_Wales', 'region_West Midlands Region', 'region_Yorkshire Region', 'disability_Y']
------------------------------


In [17]:
# 6. Normalization
X_static = df_encoded[final_static_cols].values
X_ts_click = df_encoded[ts_click_cols].values
X_ts_score = df_encoded[ts_score_cols].values
X_ts_late = df_encoded[ts_late_cols].values
y = df_encoded['target'].values

# Chia tập Train/Test (Split trước khi Scale để tránh data leakage)
# Lưu ý: Ta chia index để lát nữa lấy lại đúng phần dữ liệu cho từng loại
indices = np.arange(len(y))
X_train_idx, X_test_idx, y_train, y_test = train_test_split(indices, y, test_size=0.2, random_state=42)

def scale_data(train_idx, test_idx, data):
    scaler = MinMaxScaler()
    # Fit trên train, transform trên cả train và test
    train_scaled = scaler.fit_transform(data[train_idx])
    test_scaled = scaler.transform(data[test_idx])
    return train_scaled, test_scaled

X_train_static, X_test_static = scale_data(X_train_idx, X_test_idx, X_static)
X_train_click, X_test_click   = scale_data(X_train_idx, X_test_idx, X_ts_click)
X_train_score, X_test_score   = scale_data(X_train_idx, X_test_idx, X_ts_score)
X_train_late, X_test_late     = scale_data(X_train_idx, X_test_idx, X_ts_late)

Các mô hình machine learning thường nhận đầu vào là 2D với hàng là mẫu và cột là các đặc trưng
nhưng mô hình LSTM nhận đầu vào là 3D theo chuẩn
(Samples, TimeSteps, Features)
với:
Samples: số lượng sinh viên
timesteps: độ dài chuỗi thời gian
features: tổng số thông tin tại một thời điểm cụ thể


In [18]:
# 7. TẠO MẢNG 3D CHO CNN-LSTM
# Input shape mong muốn: (Samples, TimeSteps, Features)
# Features = 3 (Click, Score, Late) + N (Static features được lặp lại)
def create_multi_channel_input(static, click, score, late, n_timesteps=40):
    n_samples = static.shape[0]

    # 1. Reshape các kênh Time-series thành (Samples, 40, 1)
    click_reshaped = click.reshape(n_samples, n_timesteps, 1)
    score_reshaped = score.reshape(n_samples, n_timesteps, 1)
    late_reshaped  = late.reshape(n_samples, n_timesteps, 1)

    # 2. Xử lý Static: Lặp lại (Tile) cho mỗi bước thời gian
    # Chuyển từ (Samples, n_static) -> (Samples, 1, n_static) -> (Samples, 40, n_static)
    static_reshaped = static.reshape(n_samples, 1, -1)
    static_tiled = np.tile(static_reshaped, (1, n_timesteps, 1))

    # 3. Ghép tất cả lại theo trục Features (axis=2)
    # Thứ tự features tại mỗi tuần: [Click, Score, Late, Static_1, Static_2, ...]
    final_input = np.concatenate([click_reshaped, score_reshaped, late_reshaped, static_tiled], axis=2)

    return final_input

# Tạo input cuối cùng
X_train_final = create_multi_channel_input(X_train_static, X_train_click, X_train_score, X_train_late)
X_test_final  = create_multi_channel_input(X_test_static, X_test_click, X_test_score, X_test_late)

print("-" * 30)
print("DỮ LIỆU ĐÃ SẴN SÀNG CHO MÔ HÌNH CNN-LSTM")
print(f"X_train shape: {X_train_final.shape} (Samples, TimeSteps, Features)")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape:  {X_test_final.shape}")
print(f"Số lượng Features tại mỗi bước thời gian: {X_train_final.shape[2]}")
print("-" * 30)

------------------------------
DỮ LIỆU ĐÃ SẴN SÀNG CHO MÔ HÌNH CNN-LSTM
X_train shape: (26074, 40, 22) (Samples, TimeSteps, Features)
y_train shape: (26074,)
X_test shape:  (6519, 40, 22)
Số lượng Features tại mỗi bước thời gian: 22
------------------------------


## Train mode thui

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

# 1. Cấu hình tham số (Dựa trên shape dữ liệu của bạn)
n_timesteps = 40  # Số tuần
n_features = 22   # Số lượng đặc trưng (Click, Score, Late, Static...)

# 2. Khởi tạo mô hình Sequential
model = Sequential()

# --- Layer 1: LSTM ---
# units=100: Số lượng tế bào nhớ (theo tài liệu PDF mục 3.5 dùng 100 units)
# return_sequences=False: Chỉ lấy output của bước thời gian cuối cùng (để phân loại)
# input_shape: Kích thước dữ liệu đầu vào
model.add(Input(shape=(n_timesteps, n_features)))
model.add(LSTM(units=100, return_sequences=False))

# --- Layer 2: Dropout ---
# Giúp chống Overfitting (Mô hình học vẹt)
# rate=0.5: Tắt ngẫu nhiên 50% nơ-ron
model.add(Dropout(0.5))

# --- Layer 3: Output (Dense) ---
# units=1: Chỉ cần 1 nơ-ron vì là Binary Classification (Đậu/Trượt)
# activation='sigmoid': Đưa ra xác suất từ 0 đến 1 ( >0.5 là Đậu, <=0.5 là Trượt)
model.add(Dense(units=1, activation='sigmoid'))

# 3. Compile mô hình
# Optimizer: Adam với learning rate 0.001 (theo tài liệu PDF [cite: 209])
# Loss: 'binary_crossentropy' (Chuẩn cho bài toán 2 lớp 0/1)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# In tóm tắt mô hình
model.summary()

# 4. Huấn luyện (Training)
# epochs=10, batch_size=32 (Theo tài liệu PDF [cite: 210])
history = model.fit(
    X_train_final, y_train,
    validation_data=(X_test_final, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100)            │        49,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,301 (192.58 KB)

 Trainable params: 49,301 (192.58 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 34s 37ms/step - accuracy: 0.8216 - loss: 0.3818 - precision: 0.7898 - recall: 0.8516 - val_accuracy: 0.9106 - val_loss: 0.2352 - val_precision: 0.8664 - val_recall: 0.9564
Epoch 2/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.9185 - loss: 0.2282 - precision: 0.8745 - recall: 0.9665 - val_accuracy: 0.9121 - val_loss: 0.2303 - val_precision: 0.8564 - val_recall: 0.9757
Epoch 3/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 29s 36ms/step - accuracy: 0.9167 - loss: 0.2251 - precision: 0.8718 - recall: 0.9660 - val_accuracy: 0.9127 - val_loss: 0.2266 - val_precision: 0.8578 - val_recall: 0.9751
Epoch 4/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 39s 34ms/step - accuracy: 0.9198 - loss: 0.2188 - precision: 0.8750 - recall: 0.9692 - val_accuracy: 0.9144 - val_loss: 0.2239 - val_precision: 0.8633 - val_recall: 0.9708
Epoch 5/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9208 - loss: 0.2145 - precision: 0.8771 - recall: 0.9694 - val_accuracy: 0.9143 - va

Mô hình hoạt động ổn định và có hiệu suất cao ngay cả khi chỉ là mô hình cơ bản (Baseline). Không có dấu hiệu bị Overfitting nặng.

    Training Accuracy: ~92.52%

    Validation Accuracy: ~91.70%

    Gap (Khoảng cách): Rất nhỏ (< 1%). Điều này chứng tỏ mô hình tổng quát hóa tốt, không bị học vẹt trên tập train.
Phân tích chi tiết:
A. Accuracy:
Kết quả đạt 91.7% trên tập validation, thấp hơn số với bài báo gốc (97.62%), thấp hơn khoảng 6%
=> Có thể liên quan đến việc áp dụng SMOTE và Feature Selection
B. Loss (Hàm mất mát) -

    Train Loss: Giảm đều từ 0.38 xuống 0.19.

    Val Loss: Giảm từ 0.23 xuống ~0.20.

    Hiện tượng: Val Loss bắt đầu đi ngang (plateau) từ Epoch 8 đến 10 (0.21 -> 0.20). Đây là dấu hiệu mô hình đã hội tụ (converged). Nếu train tiếp, Val Loss có thể sẽ tăng lại (Overfitting). Dừng ở Epoch 10 là hợp lý.
C. Precision vs. Recall (Điểm thú vị nhất) -
    Validation Recall (97.71%): Rất cao.

        Ý nghĩa: Trong số tất cả những sinh viên thực sự Đậu (Pass/Distinction), mô hình tìm ra được 97.7%. Tức là mô hình rất hiếm khi bỏ sót sinh viên giỏi/đạt.

    Validation Precision (86.36%): Thấp hơn Recall đáng kể.
        Ý nghĩa: Trong số những sinh viên mô hình dự đoán là Đậu, chỉ có 86% là đậu thật.

        Hệ quả: Mô hình đang hơi "lạc quan". Nó có xu hướng dự đoán sinh viên sẽ Đậu nhiều hơn mức cần thiết. Điều này dẫn đến việc một số sinh viên Trượt (Fail) bị nhầm thành Đậu (False Positive).
Tại sao lại như vậy? Có thể do dữ liệu của bạn vẫn còn hơi mất cân bằng (số lượng sinh viên Pass nhiều hơn Fail), hoặc do bản chất của SMOTE làm cho ranh giới giữa Pass/Fail bị mờ nhạt theo hướng có lợi cho lớp Pass.
Trong giáo dục, mục tiêu thường là phát hiện sinh viên có nguy cơ rớt (Fail) để can thiệp sớm.

    Với Recall lớp Pass cao (97%), đồng nghĩa với việc False Negative thấp. Tức là rất ít khi mô hình báo "Rớt" cho một sinh viên "Đậu".

    Tuy nhiên, ta cần quan tâm chiều ngược lại: Liệu mô hình có bắt được hết sinh viên Rớt không? (Chỉ số này ẩn trong Accuracy và Precision). Với Precision 86%, có nghĩa là vẫn còn một lượng sinh viên Rớt bị lọt lưới (mô hình báo Đậu nhưng thực tế Rớt).

## Draft

In [ ]:
## another version of gemini (it's crazy you know)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

# ==========================================
# 1. LOAD DỮ LIỆU & LÀM SẠCH CƠ BẢN
# ==========================================
# Giả định các df đã được load như trong hình bạn gửi
# df_info, df_vle, df_assess, df_assessments, df_courses, df_regis

# Loại bỏ các bản ghi bị rút học (Withdrawn) nếu muốn tập trung vào Pass/Fail (tùy bài toán)
# Nhưng theo tài liệu, ta giữ lại và coi Fail/Withdrawn là lớp 0 (Weak).
# Encode Target: Distinction/Pass -> 1, Fail/Withdrawn -> 0
df_info['target'] = df_info['final_result'].apply(lambda x: 1 if x in ['Distinction', 'Pass'] else 0)

# ==========================================
# 2. XỬ LÝ DỮ LIỆU TIME-SERIES (VLE CLICKS)
# ==========================================
# Mục tiêu: Tạo chuỗi lịch sử click cho mỗi sinh viên qua các tuần (Ví dụ: 39 tuần)

# Xử lý ngày âm (trước khi bắt đầu khóa) về tuần 0
df_vle['week'] = df_vle['date'].apply(lambda x: 0 if x < 0 else x // 7)
# Giới hạn số tuần (ví dụ OULAD thường xét đến tuần 39)
df_vle = df_vle[df_vle['week'] <= 39]

# Pivot table: Hàng là SV, Cột là Tuần (week_0, week_1, ...), Giá trị là tổng click
vle_pivot = df_vle.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week',
    values='sum_click',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Đổi tên cột tuần
week_cols = [c for c in vle_pivot.columns if isinstance(c, int)]
vle_pivot.rename(columns={c: f'week_{c}' for c in week_cols}, inplace=True)
ts_features = [f'week_{i}' for i in week_cols] # Danh sách tên cột time-series

print(f"Số lượng bước thời gian (Time-steps): {len(ts_features)}")

# ==========================================
# 3. XỬ LÝ DỮ LIỆU ASSESSMENT (ĐIỂM SỐ)
# ==========================================
# Gộp bảng điểm chi tiết với thông tin bài kiểm tra
assess_full = pd.merge(df_assess, df_assessments, on='id_assessment', how='inner')

# Tính điểm trung bình (weighted score nếu cần) của mỗi SV trong mỗi khóa
# Lưu ý: Điểm số cũng có thể coi là time-series, nhưng ở đây ta lấy trung bình làm đặc trưng tĩnh (đơn giản hóa)
score_agg = assess_full.groupby(['id_student', 'code_module', 'code_presentation'])['score'].mean().reset_index()
score_agg.rename(columns={'score': 'mean_score'}, inplace=True)
# Fill NaN điểm số bằng 0 (sinh viên không nộp bài)
score_agg['mean_score'].fillna(0, inplace=True)

# ==========================================
# 4. GỘP TẤT CẢ DỮ LIỆU (MERGE)
# ==========================================
# Bắt đầu từ bảng gốc studentInfo
df_final = df_info.copy()

# Merge với dữ liệu VLE (Time-series)
df_final = pd.merge(df_final, vle_pivot, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Merge với dữ liệu điểm số
df_final = pd.merge(df_final, score_agg, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Điền 0 cho những sinh viên không có dữ liệu VLE hoặc điểm (những người bỏ học sớm/không tương tác)
df_final[ts_features] = df_final[ts_features].fillna(0)
df_final['mean_score'] = df_final['mean_score'].fillna(0)

# ==========================================
# 5. XỬ LÝ STATIC FEATURES (ENCODING)
# ==========================================
# Chọn các cột nhân khẩu học
static_cols = ['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability', 'num_of_prev_attempts', 'mean_score']
# One-Hot Encoding cho các biến categorical
df_final = pd.get_dummies(df_final, columns=['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability'], drop_first=True)

# Cập nhật danh sách static features sau khi one-hot
final_static_cols = [c for c in df_final.columns if c not in ts_features + ['id_student', 'code_module', 'code_presentation', 'final_result', 'target', 'date_registration', 'date_unregistration']]

print(f"Số lượng đặc trưng tĩnh (Static features): {len(final_static_cols)}")

# ==========================================
# 6. FEATURE SELECTION (RFE) & SPLIT
# ==========================================
# Tách X và y
# Lưu ý: RFE thường chạy trên dữ liệu 2D phẳng. Ta sẽ chọn đặc trưng tĩnh quan trọng nhất.
X_static = df_final[final_static_cols]
X_ts = df_final[ts_features] # Time series clicks
y = df_final['target']

# Chuẩn hóa (Normalization) - Theo mục 3.2
scaler_static = MinMaxScaler()
X_static_scaled = pd.DataFrame(scaler_static.fit_transform(X_static), columns=X_static.columns)

scaler_ts = MinMaxScaler()
X_ts_scaled = pd.DataFrame(scaler_ts.fit_transform(X_ts), columns=X_ts.columns)

# Chia tập Train/Test TRƯỚC KHI SMOTE để tránh data leakage
X_train_static, X_test_static, X_train_ts, X_test_ts, y_train, y_test = train_test_split(
    X_static_scaled, X_ts_scaled, y, test_size=0.2, random_state=42
)

# Áp dụng RFE cho phần Static Data (để loại bỏ bớt các biến nhân khẩu học nhiễu)
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10) # Chọn top 10 đặc trưng tĩnh
rfe.fit(X_train_static, y_train)
selected_static = X_train_static.columns[rfe.support_]
print("Top static features selected:", selected_static)

# Cập nhật lại tập dữ liệu chỉ với feature đã chọn
X_train_static = X_train_static[selected_static]
X_test_static = X_test_static[selected_static]

# ==========================================
# 7. SMOTE (XỬ LÝ MẤT CÂN BẰNG)
# ==========================================
# SMOTE cần đầu vào là 2D. Ta cần gộp static và time-series lại để sinh mẫu giả lập đồng bộ
X_train_full = pd.concat([X_train_static, X_train_ts], axis=1)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_full, y_train)

print("Shape sau SMOTE:", X_train_resampled.shape)

# Tách lại ra Static và Time-series sau khi SMOTE
X_train_static_res = X_train_resampled[selected_static]
X_train_ts_res = X_train_resampled[ts_features]

# ==========================================
# 8. RESHAPE DATA CHO CNN-LSTM (QUAN TRỌNG)
# ==========================================
# Để CNN-LSTM học được cả ngữ cảnh tĩnh và động, cách phổ biến là:
# "Repeat Vector": Lặp lại thông tin tĩnh cho mỗi bước thời gian và ghép vào chuỗi click.
# Input shape cuối cùng: (Samples, TimeSteps, Features)
# Features = 1 (click) + N (static features)

def create_3d_input(static_df, ts_df):
    samples = static_df.shape[0]
    timesteps = ts_df.shape[1]

    # Chuyển time-series thành (Samples, TimeSteps, 1)
    ts_array = ts_df.values.reshape(samples, timesteps, 1)

    # Chuyển static thành (Samples, 1, n_static) rồi lặp lại cho các timesteps
    static_array = static_df.values.reshape(samples, 1, -1)
    static_tiled = np.tile(static_array, (1, timesteps, 1)) # (Samples, TimeSteps, n_static)

    # Ghép lại: Tại mỗi tuần, ta có [số click, giới tính, vùng miền, điểm số...]
    return np.concatenate([ts_array, static_tiled], axis=2)

X_train_final = create_3d_input(X_train_static_res, X_train_ts_res)
# X_test không SMOTE, dùng dữ liệu gốc
X_test_final = create_3d_input(X_test_static, X_test_ts)

print("="*30)
print(f"Dữ liệu hoàn thiện sẵn sàng cho mô hình!")
print(f"Train shape: {X_train_final.shape} (Samples, TimeSteps, Features)")
print(f"Test shape:  {X_test_final.shape}")
print(f"Labels shape: {y_train_resampled.shape}")